In [59]:

!git clone https://github.com/karpathy/minGPT.git  -q
%cd minGPT
!pip install -e . -qqq
!pip install transformers[torch] -qqq

c:\Users\au616584\OneDrive - Aarhus Universitet\Datalogi\PhD\Faster-GPT-generation\minGPT\minGPT


fatal: destination path 'minGPT' already exists and is not an empty directory.
ERROR: file:///C:/Users/au616584/OneDrive%20-%20Aarhus%20Universitet/Datalogi/PhD/Faster-GPT-generation/minGPT/minGPT does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [60]:
import torch
from mingpt.model import GPT
print("step")
from transformers import GPT2Tokenizer, GPT2LMHeadModel
print("step 2")
from mingpt.utils import set_seed
print("step 3")
from mingpt.bpe import BPETokenizer
set_seed(3407)
print("step 4")
model_type = 'gpt2-large' 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPT.from_pretrained(model_type)
model.to(device)
model.eval()

step
step 2
step 3
step 4
number of parameters: 774.03M


c:\Users\au616584\Anaconda3\envs\minGPT\lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\au616584\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=1280, out_features=3840, bias=True)
          (c_proj): Linear(in_features=1280, out_features=1280, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear(in_features=1280, out_features=5120, bias=True)
          (c_proj): Linear(in_features=5120, out_features=1280, bias=True)
          (act): NewGELU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      

In [61]:
from mingpt.bpe import BPETokenizer
tokenizer = BPETokenizer()

num_samples = 1

x   = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long)
x   = x.expand(num_samples, -1).to(device)
y   = model.generate(x, max_new_tokens=3, do_sample=True, top_k=40)
out = tokenizer.decode(y[0].cpu().squeeze())

print(out)

Hi)




In [62]:
from torch.nn import functional as F
import torch as T

def sample_tokens(logits: T.Tensor, z: T.Tensor) -> T.Tensor:
    cdf = T.cumsum(T.nn.functional.softmax(logits, dim=1), dim=1)
    return (cdf < z[:, None]).sum(1) # holy fucking douglas

def unsample_z(logits: T.Tensor, tokens: T.Tensor) -> T.Tensor:
    cdf = T.cumsum(T.cat([T.zeros_like(logits[:, 0, None]), T.nn.functional.softmax(logits, dim=1)], dim=1), dim=1)
    idx = T.arange(tokens.shape[0])
    return T.stack([cdf[idx, tokens], cdf[idx, tokens+1]])



@torch.no_grad()
def generate(self, idx, Z, max_new_tokens):

        for i in range(max_new_tokens):
            logits, _ = self(idx)
            logits    = logits[:, -1, :] 
            idx_next  = sample_tokens(logits, Z[i].view(1,) ).view(1,1)

            redo = unsample_z(logits, idx_next)
            assert redo[0] <= Z[i] <=  redo[1]

            idx       = torch.cat((idx, idx_next), dim=1)
            #print(redo[0], Z[i], redo[1])

        return idx

# generate
max_new_tokens = 2
n = 50257

torch.manual_seed(42)
Z = torch.rand((max_new_tokens), device=device)

num_samples = 1
x = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long)
x = x.expand(num_samples, -1).to(device)

y = generate(model, x, Z, max_new_tokens=max_new_tokens)#, top_k=40)
out = tokenizer.decode(y[0].cpu().squeeze())
print(out)

# y = generate(model, x, Z, max_new_tokens=max_new_tokens)#, top_k=40)
# out = tokenizer.decode(y[0].cpu().squeeze())
# print(out)

logits, _ = model(y)
print(logits.shape, y.shape)
#idx_next  = sample_tokens(logits, Z.view(1,) ).view(1,1)

logits, _ = model(y)
redo = unsample_z(logits[0, 0:-1], y[0, 1:]) # this is the inverse!  F(logits, y) 
print(redo)
print(Z)

Hiboys Theory
torch.Size([1, 3, 50257]) torch.Size([1, 3])
tensor([[0.88218599557876586914, 0.91500407457351684570],
        [0.88228327035903930664, 0.91504842042922973633]],
       grad_fn=<StackBackward0>)
tensor([0.88226926326751708984, 0.91500395536422729492])


In [63]:
# torch.manual_seed(11)
# Z = torch.rand((max_new_tokens), device=device)
# print(f"Z_j = {Z[0]}")
# print(f"Z = {Z}")
# x = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long)
# print(x)
# y_z = generate(model, x, Z, max_new_tokens=max_new_tokens)#, top_k=40)
# print(y_z)
# print(tokenizer.decode(y_z[0].cpu().squeeze()))
# x_ = x.clone()
# for p in range(max_new_tokens):
#     for i in range(n):
#         print(i, end='\r')
#         x_prime = torch.cat((x_, torch.tensor([[i]], dtype=torch.long)), dim=1)
#         # x_prime = torch.cat((x_prime, torch.tensor([[i]], dtype=torch.long)), dim=1)
#         logits, _ = model(x_prime)
#         redo = unsample_z(logits[0, 0:-1], x_prime[0, 1:]) # this is the inverse!  F(logits, y)
#         # print(redo[0], Z[p], redo[1][p],end='\r')
#         if redo[0][p] <= Z[p] <= redo[1][p]:
#             print(i, redo[0], Z, redo[1])
#             x_ = x_prime
#             break
#This is doing the same as sample tokens!!!

In [64]:
#do the same, but iterate over everything in redo to find the right one
torch.manual_seed(12)
max_new_tokens = 2
Z = torch.rand((max_new_tokens), device=device)
print(f"Z_j = {Z[0]}")
print(f"Z = {Z}")
x = torch.tensor([tokenizer.encoder.encode('20 plus 1 equals')], dtype=torch.long)
print(x)
y_z = generate(model, x, Z, max_new_tokens=max_new_tokens)#, top_k=40)
print(y_z)
print(tokenizer.decode(y_z[0].cpu().squeeze()))
torch.manual_seed(11)

x_ = x.clone()
for p in range(max_new_tokens):
        # x_prime = torch.cat((x_prime, torch.tensor([[i]], dtype=torch.long)), dim=1)
    logits, _ = model(x_)
    next_word = sample_tokens(logits[:, -1, :], Z[p].view(1,) ).view(1,1)
    print(next_word)
    x_ = torch.cat((x_, next_word), dim=1)    
        # redo = unsample_z(logits[0, 0:-1], x_prime[0, 1:]) # this is the inverse!  F(logits, y)
        # # print(redo[0], Z[p], redo[1][p],end='\r')
        # if redo[0][p] <= Z[p] <= redo[1][p]:
        #     print(i, redo[0], Z, redo[1])
        #     x_ = x_prime
        #     break
print(tokenizer.decode(x_[0].cpu().squeeze()))

Z_j = 0.46568745374679565
Z = tensor([0.46568745374679565430, 0.23276680707931518555])
tensor([[ 1238,  5556,   352, 21767]])
tensor([[ 1238,  5556,   352, 21767,   860,    13]])
20 plus 1 equals 9.
tensor([[860]])
tensor([[13]])
20 plus 1 equals 9.


In [65]:
torch.manual_seed(11)
Z = torch.rand((1), device=device)
print(f"Z_j = {Z[0]}")
print(f"Z = {Z}")
x = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long)
print(x)
x_ = x.clone()
for i in range(550,551):
    print(i, end='\r')
    x_prime = torch.cat((x_, torch.tensor([[tokenizer.encoder.encoder['.']]], dtype=torch.long)), dim=1)
    # x_prime = torch.cat((x_prime, torch.tensor([[i]], dtype=torch.long)), dim=1)
    logits, _ = model(x_prime)
    redo = unsample_z(logits[0, 0:-1], x_prime[0, 1:]) # this is the inverse!  F(logits, y)
    # print(redo[0], Z[p], redo[1][p],end='\r')
    if redo[0] <= Z <= redo[1]:
        print(i, redo[0], Z, redo[1])
        x_ = x_prime
        break


Z_j = 0.14904171228408813
Z = tensor([0.14904171228408813477])
tensor([[17250]])


In [66]:
torch.manual_seed(45)


max_new_tokens = 5
n = 50257
Z = torch.rand((max_new_tokens), device=device)

num_samples = 1
x = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long)
x = x.expand(num_samples, -1).to(device)
y = generate(model, x, Z, max_new_tokens=max_new_tokens)#, top_k=40)
print(f"y: {y}")
out = tokenizer.decode(y[0].cpu().squeeze())
print(out)

logits, _ = model(y) 
print(logits.shape)
inverse_ranges = unsample_z(logits[0,0:-1], y[0,1:])
assert(torch.all(torch.logical_and(inverse_ranges[0]<=Z, Z<=inverse_ranges[1])))
print(f"Z:{Z}, inverse range: {inverse_ranges}")
#Test that a different number in the interval is mapped to the same value
Z_prime = inverse_ranges[0]+0.000001
# print(Z_prime)
y_prime = generate(model, x, Z_prime, max_new_tokens = max_new_tokens)
out_prime = tokenizer.decode(y_prime[0].cpu().squeeze())
print(out_prime)

y: tensor([[17250,    11, 24977,   475,  6954,   286]])
Hi, 1949 but evolution of
torch.Size([1, 6, 50257])
Z:tensor([0.18694752454757690430, 0.96132838726043701172, 0.68344897031784057617,
        0.89879584312438964844, 0.05050849914550781250]), inverse range: tensor([[0.08008711040019989014, 0.96132808923721313477, 0.68332219123840332031,
         0.89879155158996582031, 0.02872197888791561127],
        [0.29271295666694641113, 0.96132868528366088867, 0.68355798721313476562,
         0.89879667758941650391, 0.08397702127695083618]],
       grad_fn=<StackBackward0>)
Hi, Omar," proves a


In [67]:
#Test how changing a single word affects the ranges
y_lwc = y.clone() #last word changed
y_lwc[0][-1] = y_lwc[0][-1]+10
print(y_lwc)
out_lwc = tokenizer.decode(y_lwc[0].cpu().squeeze())
print(out_lwc)
logits_lwc,_ = model(y_lwc)
inverse_ranges_lwc = unsample_z(logits_lwc[0,0:-1], y_lwc[0, 1:])
print(inverse_ranges_lwc)
#notice that in this case all the previous words are not changed. So you could do it element by element from the back?
y_fwc = y.clone() #last word changed
y_fwc[0][1] = y_fwc[0][1]+20
print(y_fwc)
out_fwc = tokenizer.decode(y_fwc[0].cpu().squeeze())
print(out_fwc)
logits_fwc,_ = model(y_fwc)
inverse_ranges_fwc = unsample_z(logits_fwc[0,0:-1], y_fwc[0, 1:])
print(inverse_ranges_fwc)
y_fwc[0][1] = y_fwc[0][1]+21
print(y_fwc)
out_fwc = tokenizer.decode(y_fwc[0].cpu().squeeze())
print(out_fwc)
logits_fwc,_ = model(y_fwc)
inverse_ranges_fwc = unsample_z(logits_fwc[0,0:-1], y_fwc[0, 1:])
print(inverse_ranges_fwc)
y_fwc[0][1] = y_fwc[0][1]+22
print(y_fwc)
out_fwc = tokenizer.decode(y_fwc[0].cpu().squeeze())
print(out_fwc)
logits_fwc,_ = model(y_fwc)
inverse_ranges_fwc = unsample_z(logits_fwc[0,0:-1], y_fwc[0, 1:])
print(inverse_ranges_fwc)
#Changing the first word changes all the probabilities of the subsequent words.

tensor([[17250,    11, 24977,   475,  6954,   296]])
Hi, 1949 but evolutionom
tensor([[0.08008711040019989014, 0.96132808923721313477, 0.68332219123840332031,
         0.89879155158996582031, 0.12114837020635604858],
        [0.29271295666694641113, 0.96132868528366088867, 0.68355798721313476562,
         0.89879667758941650391, 0.12114951014518737793]],
       grad_fn=<StackBackward0>)
tensor([[17250,    31, 24977,   475,  6954,   286]])
Hi@ 1949 but evolution of
tensor([[0.44990682601928710938, 0.83459746837615966797, 0.52469068765640258789,
         0.89261054992675781250, 0.03333691507577896118],
        [0.45155003666877746582, 0.83459752798080444336, 0.52479505538940429688,
         0.89261370897293090820, 0.15026678144931793213]],
       grad_fn=<StackBackward0>)
tensor([[17250,    52, 24977,   475,  6954,   286]])
HiU 1949 but evolution of
tensor([[0.47763058543205261230, 0.95056873559951782227, 0.56360703706741333008,
         0.88898706436157226562, 0.02912090905010700226],
 

In [68]:
for i in range(20):
    y_fwc = y.clone() #last word changed
    y_fwc[0][1] = y_fwc[0][1]+i
    print(y_fwc)
    out_fwc = tokenizer.decode(y_fwc[0].cpu().squeeze())
    print(out_fwc)
    logits_fwc,_ = model(y_fwc)
    inverse_ranges_fwc = unsample_z(logits_fwc[0,0:-1], y_fwc[0, 1:])
    print(inverse_ranges_fwc[0][1], inverse_ranges_fwc[1][1])

tensor([[17250,    11, 24977,   475,  6954,   286]])
Hi, 1949 but evolution of
tensor(0.96132808923721313477, grad_fn=<SelectBackward0>) tensor(0.96132868528366088867, grad_fn=<SelectBackward0>)
tensor([[17250,    12, 24977,   475,  6954,   286]])
Hi- 1949 but evolution of
tensor(0.89590907096862792969, grad_fn=<SelectBackward0>) tensor(0.89590936899185180664, grad_fn=<SelectBackward0>)
tensor([[17250,    13, 24977,   475,  6954,   286]])
Hi. 1949 but evolution of
tensor(0.96130990982055664062, grad_fn=<SelectBackward0>) tensor(0.96131056547164916992, grad_fn=<SelectBackward0>)
tensor([[17250,    14, 24977,   475,  6954,   286]])
Hi/ 1949 but evolution of
tensor(0.82585006952285766602, grad_fn=<SelectBackward0>) tensor(0.82585090398788452148, grad_fn=<SelectBackward0>)
tensor([[17250,    15, 24977,   475,  6954,   286]])
Hi0 1949 but evolution of
tensor(0.93908935785293579102, grad_fn=<SelectBackward0>) tensor(0.93909096717834472656, grad_fn=<SelectBackward0>)
tensor([[17250,    16, 24

In [69]:
def nn(x):   
  return torch.argmin( torch.sum( (model.transformer.wte.weight.reshape(1, -1, 1024) - x.reshape(-1, 1, 1024))**2, axis=2), axis=1)

def extracted_forward(embedding, t, device):
  pos    = torch.arange(0, t, dtype=torch.long, device=device).unsqueeze(0) # shape (1, t)
  # the rest is just normal forward pass 
  pos_emb = model.transformer.wpe(pos) # position embeddings of shape (1, t, n_embd)
  #x = model.transformer.drop(tok_emb + pos_emb)
  x = embedding + pos_emb
  for block in model.transformer.h:
      x = block(x)
  x = model.transformer.ln_f(x)
  return model.lm_head(x) # output of forward pass 
  
torch.manual_seed(42)
max_new_tokens, n = 5, 50257
Z = torch.rand((max_new_tokens), device=device)
prompt  = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long, device=device)
generated_target = generate(model, prompt, Z, max_new_tokens = max_new_tokens)
target_idx = generated_target
b, t    = target_idx.size()

print("target_idx: ", target_idx)
logits, _ = model(target_idx)
print("unsampled: ", unsample_z(logits[0,0:-1], target_idx[0,1:]))
print("unsampled: ", unsample_z(logits[0,1:], target_idx[0,1:]))
print("Z:", Z)
print(generated_target)


target_emb = model.transformer.wte(target_idx)
target_emb_grad = target_emb.clone().detach().requires_grad_(True) # then pick idx to be nearest neighboar to token emb! 
print(f"target sentence: ", tokenizer.decode(target_idx.cpu().squeeze()))


scrambled_idx = target_idx.clone()
scrambled_idx[0][-2] = scrambled_idx[0][-2]+2
print(f"scrambled idx, {scrambled_idx}")
print(f"scrambled sentence: ", tokenizer.decode(scrambled_idx.cpu().squeeze()))
scrambled_emb = model.transformer.wte(scrambled_idx)
scrambled_emb_grad = scrambled_emb.clone().detach().requires_grad_(True)

adam = torch.optim.Adam([scrambled_emb_grad], lr=0.01) 
for i in range(1): 
  idx_guess = nn(scrambled_emb_grad).view(-1)
  adam.zero_grad()
   # just do forward pass 
  W_logits = extracted_forward(scrambled_emb_grad, t, device) # output of forward pass 
  enc_W_logits = unsample_z(W_logits[0,0:-1], idx_guess[1:] )
  # prompt = torch.tensor([[tokenizer.encoder.encoder['What']]], dtype=torch.long, device=device)
  # y_prime = generate(model, prompt, (enc_W_logits[0]+0.0001), max_new_tokens = max_new_tokens)
  # out_prime = tokenizer.decode(y_prime[0].cpu().squeeze())
  loss = torch.mean((Z - enc_W_logits[0]).abs()) +  torch.mean((Z - enc_W_logits[1]).abs())

  # loss = (Z[-1] - enc_W_logits[0][-1])**2+ (Z[-1] - enc_W_logits[1][-1])**2
  print(f"min: {enc_W_logits[0]}, max: {enc_W_logits[1]}")

  print(scrambled_emb_grad[0][4])
  print("\r[%i / %i] "%(i+1, 10000),
        torch.mean((target_emb_grad[0][-2]- scrambled_emb_grad[0][-2]).abs()).data.cpu().numpy(), 
        loss.data.cpu().numpy(), 
        torch.set_printoptions(precision=20),
        print("full max_min: ", enc_W_logits[0][-2], enc_W_logits[1][-2]),
        # idx_guess.cpu().numpy(), 
        tokenizer.decode(idx_guess.cpu().squeeze()), end="")

  if i == 0: print("")
  loss.backward()
  # assert(torch.all(scrambled_emb_grad.grad[0][-1]!=0))
  scrambled_emb_grad.grad[:,:-2,:]=0
  # assert(torch.any(scrambled_emb_grad.grad[0][-1]!=0))  
  adam.step()
  adam.zero_grad()

target_idx:  tensor([[17250, 13202, 17003,   198,   198,  1525]])
unsampled:  tensor([[0.88218599557876586914, 0.91500401496887207031, 0.33725765347480773926,
         0.00238766125403344631, 0.37818861007690429688],
        [0.88228327035903930664, 0.91504836082458496094, 0.43607437610626220703,
         0.99255758523941040039, 0.39490467309951782227]],
       grad_fn=<StackBackward0>)
unsampled:  tensor([[0.89036899805068969727, 0.93998485803604125977, 0.00238766125403344631,
         0.19141931831836700439, 0.33521190285682678223],
        [0.89038985967636108398, 0.94103837013244628906, 0.99255758523941040039,
         0.19146537780761718750, 0.33521443605422973633]],
       grad_fn=<StackBackward0>)
Z: tensor([0.88226926326751708984, 0.91500395536422729492, 0.38286375999450683594,
        0.95930564403533935547, 0.39044821262359619141])
tensor([[17250, 13202, 17003,   198,   198,  1525]])
target sentence:  Hiboys Theory

by
scrambled idx, tensor([[17250, 13202, 17003,   198,   200

RuntimeError: shape '[1, -1, 1024]' is invalid for input of size 64328960

In [ ]:
#Using the fact that we have a limited search space, can we just iterate over all the possible words and find the one that minimizes the loss?
# torch.manual_seed(42)
max_new_tokens, n = 5, 50257
Z = torch.rand((max_new_tokens), device=device)
prompt  = torch.tensor([[tokenizer.encoder.encoder['Hi']]], dtype=torch.long, device=device)
generated_target = generate(model, prompt, Z, max_new_tokens = max_new_tokens)
target_idx = generated_target
b, t    = target_idx.size()

print("target_idx: ", target_idx)
logits, _ = model(target_idx)
print("unsampled: ", unsample_z(logits[0,0:-1], target_idx[0,1:]))
print("Z:", Z)
print(generated_target)

range_shuffled = torch.randperm(n)

scrambled_idx = target_idx.clone()
scrambled_idx[0][-2] = scrambled_idx[0][-2]+2
for i in range(1,max_new_tokens+1):
    logits_fwc,_ = model(scrambled_idx)
    inverse_ranges_fwc = unsample_z(logits_fwc[0,0:-1], scrambled_idx[0, 1:])
    print(inverse_ranges_fwc[0][i-1], inverse_ranges_fwc[1][i-1])
    # if Z[i-1] > inverse_ranges_fwc[0][i-1] and Z[i-1] < inverse_ranges_fwc[1][i-1]:
    #     print(Z[i])
    #     print("Z[i] < inverse_ranges_fwc[0][i]")
    #     continue
    print(inverse_ranges_fwc[0][i-1], inverse_ranges_fwc[1][i-1])
    for j in range_shuffled:
        print("in the loop with id: ", i)
        scrambled_idx[0][i] = j
        logits_fwc,_ = model(scrambled_idx)
        inverse_ranges_fwc = unsample_z(logits_fwc[0,0:-1], scrambled_idx[0, 1:])
        if Z[i-1] < inverse_ranges_fwc[1][i-1] or Z[i-1] > inverse_ranges_fwc[0][i-1]:
            print("Z[i] < inverse_ranges_fwc[0][i]")
            scrambled_idx[0][i-1] = j
            break
print(tokenizer.decode(scrambled_idx.cpu().squeeze()))
print(scrambled_idx)



torch.Size([1, 50257]) torch.Size([1])
torch.Size([1, 50257]) torch.Size([1])
torch.Size([1, 50257]) torch.Size([1])
torch.Size([1, 50257]) torch.Size([1])
torch.Size([1, 50257]) torch.Size([1])
target_idx:  tensor([[17250,    12,  1596,  4064, 23851,   198]])
unsampled:  tensor([[0.38414323329925537109, 0.25656664371490478516, 0.79335814714431762695,
         0.94075173139572143555, 0.11056133359670639038],
        [0.62366670370101928711, 0.25657364726066589355, 0.79413223266601562500,
         0.94077730178833007812, 0.16156536340713500977]],
       grad_fn=<StackBackward0>)
Z: tensor([0.60089534521102905273, 0.25657248497009277344, 0.79364132881164550781,
        0.94077146053314208984, 0.13318592309951782227])
tensor([[17250,    12,  1596,  4064, 23851,   198]])
tensor(0.38414323329925537109, grad_fn=<SelectBackward0>) tensor(0.62366670370101928711, grad_fn=<SelectBackward0>)
tensor(0.38414323329925537109, grad_fn=<SelectBackward0>) tensor(0.62366670370101928711, grad_fn=<SelectBa

In [ ]:
###download reuters dataset and add create a pandas dataframe with the documents as entries
import nltk
nltk.download('reuters')
from nltk.corpus import reuters
import pandas as pd

# Get the list of document IDs and categories
doc_ids = reuters.fileids()
doc_categories = [reuters.categories(fileid) for fileid in doc_ids]

# Load the documents into a DataFrame
data = []
for doc_id, categories in zip(doc_ids, doc_categories):
    text = reuters.raw(doc_id)
    data.append({'doc_id': doc_id, 'categories': categories, 'text': text})
df = pd.DataFrame(data)


[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\au616584\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [ ]:
import nltk
nltk.download('reuters')


[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\au616584\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


True

In [ ]:
from nltk.corpus import reuters
import random
# Print the number of categories in the dataset
print("Number of categories: ", len(reuters.categories()))

# Print the number of documents in the dataset
print("Number of documents: ", len(reuters.fileids()))

# Print the text of the first document in the dataset
print("First document text: ", reuters.raw(reuters.fileids()[1]))

Number of categories:  90
Number of documents:  10788
First document text:  CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STOCKS
  A survey of 19 provinces and seven cities
  showed vermin consume between seven and 12 pct of China's grain
  stocks, the China Daily said.
      It also said that each year 1.575 mln tonnes, or 25 pct, of
  China's fruit output are left to rot, and 2.1 mln tonnes, or up
  to 30 pct, of its vegetables. The paper blamed the waste on
  inadequate storage and bad preservation methods.
      It said the government had launched a national programme to
  reduce waste, calling for improved technology in storage and
  preservation, and greater production of additives. The paper
  gave no further details.
  




In [ ]:
from datasets import load_dataset
dataset = load_dataset("anthropic/hh-rlhf")




























































Extracting data files: 100%|██████████| 2/2 [00:04<00:00,  2.46s/it]


Dataset json downloaded and prepared to C:/Users/au616584/.cache/huggingface/datasets/anthropic___json/anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00,  6.78it/s]


In [ ]:
import random
import torch
from datasets import load_dataset
import pickle
vocab_size = 50257

def corrupt_sentence(sentence, k, tokenizer, device):
    sentence_encoded = torch.tensor([[tokenizer.encoder.encode(sentence)]], dtype=torch.long, device=device)
    corruption_indices = random.sample(range(0, len(sentence_encoded[0][0])), k)
    corrupted_encoding = sentence_encoded.clone().detach()

    for j in corruption_indices:
        corrupted_encoding[0][0][j] = (random.sample(range(0, vocab_size), 1)[0])

    corrupted_sentence = tokenizer.decode(corrupted_encoding[0].cpu().squeeze())
    return corrupted_sentence

def save_sentences(sentences, filename):
    with open(filename, 'wb') as f:
        pickle.dump(sentences, f)

def load_sentences(filename):
    with open(filename, 'rb') as f:
        sentences = pickle.load(f)
    return sentences

dataset = load_dataset("anthropic/hh-rlhf")

# Set the corruption parameter
k = 10

# Apply corruption to the dataset
original_and_corrupted_sentences = []

for i, example in enumerate(dataset['train']):
    if i >= 10:
        break
    original_sentence = example['chosen']
    corrupted_sentence = corrupt_sentence(original_sentence, k, tokenizer, device)
    original_and_corrupted_sentences.append((original_sentence, corrupted_sentence))

# Save the sentences
save_sentences(original_and_corrupted_sentences, 'corrupted_sentences.pkl')

# Load the sentences
loaded_sentences = load_sentences('corrupted_sentences.pkl')

# Print an example
print(dataset['train'][1]['chosen'])
print(loaded_sentences[1][0][:1000])
print("**************")
print(loaded_sentences[1][1])

Found cached dataset json (C:/Users/au616584/.cache/huggingface/datasets/anthropic___json/anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 2/2 [00:00<00:00, 40.02it/s]



Human: What kind of noises did dinosaurs make?

Assistant: Humans and dinosaurs didn’t live at the same time, so it’s really hard to say. The best place to find out what noises dinosaurs made would be

Human: yes they did

Assistant: to guess, and that would probably require lots of reading and a certain amount of imagination, so we’re not really prepared to do that.

Human: you cant read

Assistant: You can read?


Human: What kind of noises did dinosaurs make?

Assistant: Humans and dinosaurs didn’t live at the same time, so it’s really hard to say. The best place to find out what noises dinosaurs made would be

Human: yes they did

Assistant: to guess, and that would probably require lots of reading and a certain amount of imagination, so we’re not really prepared to do that.

Human: you cant read

Assistant: You can read?
**************

 PleasantHuman: What kind of noises did dinosaurs make?

Assistant: Humansimize backdrop didn’t live at the same time, so it�INTs really hard to

In [ ]:
import random
import torch
import re
from datasets import load_dataset
from transformers import AutoTokenizer
import pickle

def corrupt_sentence(sentence, k, tokenizer, device):
    sentence_encoded = torch.tensor([[tokenizer.encoder.encode(sentence)]], dtype=torch.long, device=device)
    corruption_indices = random.sample(range(0, len(sentence_encoded[0][0])), k)
    corrupted_encoding = sentence_encoded.clone().detach()

    for j in corruption_indices:
        corrupted_encoding[0][0][j] = (random.sample(range(0, vocab_size), 1)[0])

    corrupted_sentence = tokenizer.decode(corrupted_encoding[0].cpu().squeeze())
    return corrupted_sentence

def save_sentences(sentences, filename):
    with open(filename, 'wb') as f:
        pickle.dump(sentences, f)

def load_sentences(filename):
    with open(filename, 'rb') as f:
        sentences = pickle.load(f)
    return sentences

dataset = load_dataset("anthropic/hh-rlhf")

# Set the corruption parameter
k = 10

# Apply corruption to the dataset
original_and_corrupted_sentences = []

for i, example in enumerate(dataset['train']):
    if i >= 10:
        break
    original_sentence = example['chosen']
    assistant_pattern = re.compile(r'Assistant:(.*?)Human:', re.DOTALL)
    assistant_response = re.search(assistant_pattern, original_sentence)
    if assistant_response:
        original_assistant_response = assistant_response.group(1)
        corrupted_assistant_response = corrupt_sentence(original_assistant_response, k, tokenizer, device)
        corrupted_sentence = original_sentence.replace(original_assistant_response, corrupted_assistant_response)
        original_and_corrupted_sentences.append((original_sentence, corrupted_sentence))

# Save the sentences
save_sentences(original_and_corrupted_sentences, 'corrupted_sentences.pkl')

# Load the sentences
loaded_sentences = load_sentences('corrupted_sentences.pkl')

# Print an example
print(dataset['train'][0]['chosen'])
print("**************")
print(loaded_sentences[0][1])


Found cached dataset json (C:/Users/au616584/.cache/huggingface/datasets/anthropic___json/anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 2/2 [00:00<00:00, 33.70it/s]



Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,

Human: What's your favorite one?

Assistant: I haven't even thought about it.
**************


Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, desktopgger, crap, fuck, shit Noble bitch, turd, shithead, shitbag, scrotum, cunt, whore, fuckBio, shit-eating, cum, cumb chemist, fucknugget, butthole, poop, jackassDifferent cocksucker, asshole, goddamn, piss, 1904, blow, wank, jism, cum-sucking, masturb ie, fagg ageing, queer, jizz, jizz-

In [ ]:
loaded_sentences = load_sentences('corrupted_sentences.pkl')
corrupted_errors = []
unchanged_errors = []
for original_sentence, original_sentence in loaded_sentences:
    

     --------------------------------------- 11.0/11.0 MB 27.3 MB/s eta 0:00:00
     ------------------------------------- 499.4/499.4 kB 30.6 MB/s eta 0:00:00


In [ ]:
import re
from datasets import load_dataset
import pickle

def save_prompts(prompts, filename):
    with open(filename, 'wb') as f:
        pickle.dump(prompts, f)

def load_prompts(filename):
    with open(filename, 'rb') as f:
        prompts = pickle.load(f)
    return prompts

dataset = load_dataset("anthropic/hh-rlhf")

# Extract the first human prompt from each sample
human_prompts = []

for i, example in enumerate(dataset['train']):
    if i >= 3:
        break
    original_sentence = example['chosen']
    human_pattern = re.compile(r'Human:(.*?)(Assistant:|$)', re.DOTALL)
    human_prompt = re.search(human_pattern, original_sentence)
    if human_prompt:
        human_prompts.append(human_prompt.group(1).strip())

# Save the human prompts
save_prompts(human_prompts, 'human_prompts.pkl')

# Load the human prompts
loaded_prompts = load_prompts('human_prompts.pkl')

# Print an example
print(loaded_prompts[0])

Found cached dataset json (C:/Users/au616584/.cache/huggingface/datasets/anthropic___json/anthropic--hh-rlhf-c8cd8dc58ab67414/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
100%|██████████| 2/2 [00:00<00:00, 25.96it/s]

What are some cuss words in english?


In [ ]:
prompts = load_prompts('human_prompts.pkl')
num_tokens = 10
Z = torch.rand((num_tokens), device=device)

for prompt in prompts:
    print(prompt)
    
    encoded  = torch.tensor([tokenizer.encoder.encode(prompt)], dtype=torch.long, device=device)
    generated = generate(model, encoded, Z, max_new_tokens = num_tokens)

    print(tokenizer.decode(generated[0]))
    

What are some cuss words in english?
What are some cuss words in english? (i know that e.g.!!!!! would
What kind of noises did dinosaurs make?
What kind of noises did dinosaurs make?


spareBison: A donkey made
If you were going to steal from a convenience store, do you think it would be better in the morning or at night?
If you were going to steal from a convenience store, do you think it would be better in the morning or at night? This is part of a reason in soaaa many
